In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from scipy.stats import mode
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, PolynomialFeatures, Normalizer,MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, validation_curve, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

%matplotlib inline

In [ ]:
data_train = pd.read_csv('Train_UWu5bXk.csv')
data_test = pd.read_csv('Test_u94Q5KV.csv')
data_train['Train_Test'] = 'Train'
data_test['Train_Test'] = 'Test'

# Data Exploring

In [ ]:
data = pd.concat([data_train, data_test], ignore_index= True, sort=False)
data.head()

In [ ]:
data.info(null_counts= True)

We can notice that, Item_Weight and Outlet_Size features have null values.

In [ ]:
#Identifying Categorical Features
categorical_features = data.dtypes[data.dtypes == 'object'].index.tolist()

In [ ]:
#Identifying numeric Features
numeric_features = data.dtypes[data.dtypes != 'object'].index.tolist()

In [ ]:
#Removing Identifier coulmns from categorical features.
categorical_features = [i for i in categorical_features if i not in ('Item_Identifier','Outlet_Identifier')]

In [ ]:
#Print the frequency of each category in each feature
for i in categorical_features:
    print('Frequency of each category in variable: {}'.format(i))
    print(data[i].value_counts(),'\n')

# Data Cleaning

1. In Item_Weight feature, we will replace Null values with the mean weight of each item.

In [ ]:
#Getting the mean weight for each item
item_avg_weight = data.pivot_table(values='Item_Weight',index='Item_Identifier', aggfunc = 'mean')

In [ ]:
#Binary column for items whose weight is null.
null_weight = data['Item_Weight'].isnull()

In [ ]:
data.loc[null_weight,'Item_Weight'] = data.loc[null_weight,'Item_Identifier'].apply(lambda x: item_avg_weight.loc[x])

In [ ]:
#Confirm null values are replace in Item_Weight
data.Item_Weight.isnull().any()

2. In Outlet_Size feature, we will replace null values with the most frequent Outlet_Size for the corresponding Outlet_Type

In [ ]:
#Getting the most frequent Outlet_Size for each Outlet_Type
outlet_size_mode = data.pivot_table(values='Outlet_Size', columns='Outlet_Type', aggfunc = (lambda x: mode(x).mode[0]) )

In [ ]:
#Binary column to select rows where Outlet_Size is null.
missing_size_bool = data.Outlet_Size.isnull()

In [ ]:
#Fill Outlet_Size null values with the most frequent Outlet_Size for Outlets of the same type.
data.loc[missing_size_bool,'Outlet_Size'] = data.loc[missing_size_bool, 'Outlet_Type'].apply(lambda x: outlet_size_mode[x])

In [ ]:
#Confirm no Null values in Outlet_Size
data.Outlet_Size.isnull().any()

3. Different spelling of same level name in Item_Fat_Content feature.

In [ ]:
data.Item_Fat_Content.unique()

In [ ]:
data.loc[(data.Item_Fat_Content == 'low fat') | (data.Item_Fat_Content == 'LF'),'Item_Fat_Content'] = 'Low Fat'
data.loc[(data.Item_Fat_Content == 'reg'),'Item_Fat_Content'] = 'Regular'

4. We have noticed that, items of type 'Household' have 'Item_Fat_Content' of 'Low Fat' which doesn't make sense!! So we repalce it by "Non-edible".

In [ ]:
data.loc[(data.Item_Type == 'Household'),'Item_Fat_Content'] = 'Non-edible'

In [ ]:
#Check categories of Item_Fat_Content after modifications
data.Item_Fat_Content.unique()

In [ ]:
data.to_csv('data_clean.csv', index = False)

# Feature Engineering

Item_Visibilty, for some items the Item_Visibility is Zero which doesn't make sense.

1. Items whose visibility is zero, their visibility is replaced by the average visibility of this item in all outlets.

In [ ]:
item_avg_vis = pd.pivot_table(data, index= [data.Item_Identifier], values= ['Item_Visibility'])

In [ ]:
zero_vis = data.Item_Visibility == 0

In [ ]:
data.loc[zero_vis,'Item_Visibility'] = data.loc[zero_vis,'Item_Identifier'].apply(lambda x: item_avg_vis.loc[x] )

In [ ]:
#Confirm no item visibility equals zero.
(data.Item_Visibility == 0).any()

2. Create a new feature to represent the relative importance given to each product in different outlets. The importance of a product is expressed in terms of Item_visibility. Such that item visibility is compared to the average visibility of this item across all outlets.

In [ ]:
data['visibility_ratio'] = data.apply(lambda x: x['Item_Visibility']/item_avg_vis.loc[x['Item_Identifier']],axis = 1)

3. Create new feature for number of years since outlet establishment which is more meaningful than the date of establishment.

Since this sales data for 2013, we will calculate the number of years from establishment to 2013.

In [ ]:
data.Outlet_Establishment_Year = 2013 - data.Outlet_Establishment_Year

4. Create a new feature to represent the relative price of each product with repsect to its mean price across all outlets. Since more expensive outlets may have lower sales.

In [ ]:
item_avg_price = pd.pivot_table(data, index= 'Item_Identifier', values='Item_MRP', aggfunc='mean')

In [ ]:
data['Item_MeanPrice_Ratio'] = data.apply(lambda x: x['Item_MRP']/item_avg_price.loc[x['Item_Identifier']], axis= 1)

5. Item_Weight may don't have any thing to do with sales. But, it can be used to calculate the price per weight unit of the item. The price per weight unit is more precise indicator for item cost. We expect that as value of this indicator increases, the sales may decrease. We'll call this indicator (specific_price)

In [ ]:
data['specific_price'] = data.apply(lambda x: x['Item_MRP']/x['Item_Weight'], axis= 1)

6. Create new feature to represent broader category of Item_Types, since we have 16 category and this is pretty much! we will replace it by 3 categories based on (Foods, Drinks, Unconsumables).

In [ ]:
# The first two characters in the Item_Identifier signifies each category.
data['Item_Identifier'].apply(lambda x: x[:2]).unique().tolist()

In [ ]:
data['Item_Type_combined'] = data['Item_Identifier'].apply(lambda x: x[:2])

In [ ]:
data['Item_Type_combined'] = data['Item_Type_combined'].map({'FD':'Food', 'DR':'Drinks', 'NC':'Non-Consumable'})

In [ ]:
#Items may be categorized based on the sales
# data.groupby('Item_Type')[['Item_Type','Item_Outlet_Sales']].sum().sort_values('Item_Outlet_Sales', ascending = False)

7. Convert categorical features into numeric features using LabelEncoder.

Create a new feature Outlet to convert Outlet_Identifier into numeric feature and keep the original feature.

In [ ]:
data['Outlet'] = data['Outlet_Identifier']

Convert other categorical features into numeric features.

In [ ]:
cat_covert_num = data.dtypes[data.dtypes == 'object'].index.tolist()
cat_covert_num = [i for i in cat_covert_num if i not in ('Item_Identifier','Outlet_Identifier','Item_Type','Train_Test')]

In [ ]:
le ={} #Dictionary of the label encoders for all categorical features
for col in cat_covert_num:
    labelencoder = LabelEncoder()
    le.update({col: labelencoder.fit(data[col])})
    data[col] = labelencoder.fit_transform(data[col])

### Drop unnecessary features.

In [ ]:
data.drop(columns= ['Item_Type', 'Outlet_Establishment_Year'], inplace= True)

### Getting dummies for categorical features.

In [ ]:
data = pd.get_dummies(data=data,columns=['Item_Fat_Content','Outlet_Size','Outlet_Location_Type','Outlet_Type','Item_Type_combined','Outlet'])

### Splitting data again into train and test

In [ ]:
train = data.loc[data['Train_Test'] == 'Train']
test = data.loc[data['Train_Test'] == 'Test'].reset_index()

In [ ]:
train.drop(columns=['Train_Test'], inplace= True)
test.drop(columns=['index','Train_Test','Item_Outlet_Sales'], inplace= True)

### Normalizing numerical features

In [ ]:
# Check features for which the difference between Min and Max is large.
train.describe()

In [ ]:
features_to_normalize = ['Item_Weight','Item_Visibility','Item_MRP','specific_price']

In [ ]:
norm ={} #Dictionary of the MinMax Scalers for all features to be normalized.
for i in features_to_normalize:
    scaler = MinMaxScaler()
    norm.update({i: scaler.fit(train[[i]])})
    train[[i]] = scaler.transform(train[[i]])
    if i == 'Item_Outlet_Sales':
        break
    else:    
        test[[i]] = scaler.transform(test[[i]])


### Export files as modified versions.

In [ ]:
train.to_csv('train_engineered.csv', index=False)
test.to_csv('test_engineered.csv', index=False)